In [26]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
import xgboost as xgb


In [27]:
import optuna


In [28]:
df = pd.read_csv('/Users/andrewkerr/Downloads/final_forreal.csv')
df

,Address,Lead,bathrooms,bedrooms,city,cityId,county,countyFIPS,countyId,homeType,...,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Population 18 to 24 years,Population 25 years and over,Population 25 to 34 years,Population 35 to 44 years,Population 45 to 64 years,Population 65 years and over,Income normalized
0,5460 w prentice cir,0,2.5,2.0,Denver,8179.0,Jefferson County,8031.0,2734.0,SINGLE_FAMILY,...,0.046394,0.000398,0.004198,0.040910,0.479545,0.105734,0.089655,0.166394,0.117762,433.452282
1,5164 jasper ct,0,2.0,3.0,Denver,11093.0,Denver County,8031.0,989.0,SINGLE_FAMILY,...,0.028902,0.013083,0.003093,0.074047,0.462977,0.151182,0.117735,0.138708,0.055352,294.547718
2,10364 e 60th pl,0,2.5,3.0,Denver,11093.0,Denver County,8031.0,989.0,SINGLE_FAMILY,...,0.063165,0.000444,0.005470,0.015398,0.492301,0.099922,0.174527,0.166810,0.051042,667.070539
3,6550 e mississippi ave,0,NaN,NaN,Denver,11093.0,Denver County,8031.0,989.0,CONDO,...,0.035552,0.000712,0.007833,0.044348,0.477826,0.124716,0.109866,0.131639,0.111605,361.751037
4,4364 liverpool ct,0,3.0,3.0,Denver,11093.0,Denver County,8031.0,989.0,SINGLE_FAMILY,...,0.098003,0.007880,0.005803,0.082812,0.458594,0.121158,0.107382,0.170578,0.059476,358.323651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70545,1912 fox way,0,1.0,3.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,SINGLE_FAMILY,...,0.026628,0.000000,0.000138,0.172016,0.413992,0.211845,0.058012,0.081679,0.062457,501.057082
70546,5626 donson way,0,1.0,2.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,SINGLE_FAMILY,...,0.020470,0.000000,0.008629,0.035207,0.482397,0.191741,0.080473,0.126313,0.083870,536.945032
70547,351 manion way,0,4.0,4.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,SINGLE_FAMILY,...,0.020470,0.000000,0.008629,0.035207,0.482397,0.191741,0.080473,0.126313,0.083870,536.945032
70548,3747 frazier st,0,NaN,NaN,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,HOME_TYPE_UNKNOWN,...,0.160050,0.000000,0.004072,0.501342,0.249329,0.089258,0.019010,0.069429,0.071632,350.835095


In [29]:
data = df.drop(columns= ['Address', 'city', 'cityId', 'county', 'countyFIPS', 'countyId', 'homeType', 'livingArea', 'regionString', 'state', 'lastSoldPrice', 'parcelId', 'zipcode', 'zpid', 'lotAreaValue', 'stateId'])
data


,Lead,bathrooms,bedrooms,latitude,livingAreaValue,longitude,lotSize,price,propertyTaxRate,rentZestimate,...,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Population 18 to 24 years,Population 25 years and over,Population 25 to 34 years,Population 35 to 44 years,Population 45 to 64 years,Population 65 years and over,Income normalized
0,0,2.5,2.0,39.618810,2614.0,-105.057540,9350.0,1002800.0,0.48,3255.0,...,0.046394,0.000398,0.004198,0.040910,0.479545,0.105734,0.089655,0.166394,0.117762,433.452282
1,0,2.0,3.0,39.787598,1164.0,-104.803670,8458.0,438800.0,0.44,2461.0,...,0.028902,0.013083,0.003093,0.074047,0.462977,0.151182,0.117735,0.138708,0.055352,294.547718
2,0,2.5,3.0,39.807630,1962.0,-104.866875,3600.0,833200.0,0.44,3099.0,...,0.063165,0.000444,0.005470,0.015398,0.492301,0.099922,0.174527,0.166810,0.051042,667.070539
3,0,NaN,NaN,39.696080,NaN,-104.911990,15096.0,352000.0,0.44,1999.0,...,0.035552,0.000712,0.007833,0.044348,0.477826,0.124716,0.109866,0.131639,0.111605,361.751037
4,0,3.0,3.0,39.777664,1442.0,-104.743310,4400.0,460600.0,0.44,2650.0,...,0.098003,0.007880,0.005803,0.082812,0.458594,0.121158,0.107382,0.170578,0.059476,358.323651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70545,0,1.0,3.0,40.429867,1260.0,-79.978420,1197.0,288000.0,1.62,1599.0,...,0.026628,0.000000,0.000138,0.172016,0.413992,0.211845,0.058012,0.081679,0.062457,501.057082
70546,0,1.0,2.0,40.484455,930.0,-79.946670,697.0,132700.0,1.62,1282.0,...,0.020470,0.000000,0.008629,0.035207,0.482397,0.191741,0.080473,0.126313,0.083870,536.945032
70547,0,4.0,4.0,40.462902,3334.0,-79.959850,2051.0,800000.0,1.62,5015.0,...,0.020470,0.000000,0.008629,0.035207,0.482397,0.191741,0.080473,0.126313,0.083870,536.945032
70548,0,NaN,NaN,40.428960,NaN,-79.952760,2400.0,10800.0,1.62,1477.0,...,0.160050,0.000000,0.004072,0.501342,0.249329,0.089258,0.019010,0.069429,0.071632,350.835095


In [30]:
#replace NaN values with average for rest of the chosen columns

columns_to_replace = data.columns
for col in columns_to_replace:
    mean = data[col].mean()
    data[col].fillna(mean, inplace = True)

In [31]:
X = data.drop(columns = 'Lead')
y = data['Lead']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [91]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
    }

    model1 = xgb.XGBClassifier(**params)

    model1.fit(X_train, y_train)

    y_pred = model1.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Set up Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get best parameters
best_params = study.best_params
best_accuracy = study.best_value

print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy:.2f}")

[I 2023-11-20 17:16:16,440] A new study created in memory with name: no-name-881a1ec3-2b6b-4073-ab7c-81f392106169
/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_12079/4216659997.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_12079/4216659997.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_12079/4216659997.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://

Best parameters: {'lambda': 2.920483517618857e-08, 'alpha': 0.0006676172195964581, 'max_depth': 10, 'subsample': 0.9600726850560262, 'colsample_bytree': 0.9384831504475356, 'eta': 0.056020310166156995, 'min_child_weight': 4, 'gamma': 0.4470651435937361}
Best accuracy: 0.89


In [32]:
best_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'booster': 'gbtree',
    'lambda': 2.920483517618857e-08,
    'alpha': 0.0006676172195964581,
    'max_depth': 10,
    'subsample': 0.9600726850560262,
    'colsample_bytree': 0.9384831504475356,
    'eta': 0.056020310166156995,
    'min_child_weight': 4,
    'gamma': 0.4470651435937361,
}

# Create an XGBoost classifier with the best parameters
model = xgb.XGBClassifier(**best_params)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

Accuracy: 0.89


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

In [33]:
from sklearn.metrics import confusion_matrix


conf_matrix = confusion_matrix(y_pred, y_test)


TP = conf_matrix[1, 1]
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]

FNR = FN / (FN + TP)
FNR

0.2792413066385669

In [35]:
FPR = FP / (FP + TN)
FPR

0.1030316845224527

In [36]:
feature_importance = model.feature_importances_
feature_importance

array([0.01403065, 0.01439458, 0.02667994, 0.01696041, 0.07742085,
       0.01670697, 0.016895  , 0.1046517 , 0.01478125, 0.01764663,
       0.08644622, 0.01572796, 0.04676019, 0.03222554, 0.04213754,
       0.07186159, 0.02984354, 0.02927256, 0.03180036, 0.0182828 ,
       0.02066371, 0.02643455, 0.02145186, 0.02459297, 0.09775738,
       0.08457312], dtype=float32)

In [37]:
X.columns

Index(['bathrooms', 'bedrooms', 'latitude', 'livingAreaValue', 'longitude',
       'lotSize', 'price', 'propertyTaxRate', 'rentZestimate',
       'taxAssessedValue', 'yearBuilt', 'zestimate', 'Hispanic or Latino',
       'White alone', 'Black or African American alone',
       'American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some Other Race alone', 'Population 18 to 24 years',
       'Population 25 years and over', 'Population 25 to 34 years',
       'Population 35 to 44 years', 'Population 45 to 64 years',
       'Population 65 years and over', 'Income normalized'],
      dtype='object')

In [38]:
#testing east coast combination
cities_to_select = ['Trenton', 'Pittsburgh', 'Washington']
east_coast = df[df['city'].isin(cities_to_select)]

cities_to_select2 = ['Grand Rapids', 'Denver']
not_east_coast = df[df['city'].isin(cities_to_select2)]

east_coast = east_coast.drop(columns= ['Address', 'city', 'cityId', 'county', 'countyFIPS', 'countyId', 'homeType', 'livingArea', 'regionString', 'state', 'lastSoldPrice', 'parcelId', 'zipcode', 'zpid', 'lotAreaValue', 'stateId'])
not_east_coast = not_east_coast.drop(columns= ['Address', 'city', 'cityId', 'county', 'countyFIPS', 'countyId', 'homeType', 'livingArea', 'regionString', 'state', 'lastSoldPrice', 'parcelId', 'zipcode', 'zpid', 'lotAreaValue', 'stateId'])

#replace NaN values with average for rest of the chosen columns

ctr = east_coast.columns
for col in ctr:
    mean = east_coast[col].mean()
    east_coast[col].fillna(mean, inplace = True)


#replace NaN values with average for rest of the chosen columns

for col in ctr:
    mean = not_east_coast[col].mean()
    not_east_coast[col].fillna(mean, inplace = True)


In [39]:
X1_train = east_coast.drop(columns = 'Lead')
X1_test = not_east_coast.drop(columns = 'Lead')
y1_train = east_coast['Lead']
y1_test = not_east_coast['Lead']


model1 = xgb.XGBClassifier(**best_params)

# Train the model
model1.fit(X1_train, y1_train)

# Make predictions on the test set
y1_pred = model1.predict(X1_test)

# Calculate accuracy
accuracy1 = accuracy_score(y1_test, y1_pred)
accuracy1

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

0.7884354060259594

In [40]:
#testing west coast combination
cities_to_select2 = ['Trenton', 'Pittsburgh', 'Washington']
east_coast1 = df[df['city'].isin(cities_to_select2)]

cities_to_select3 = ['Grand Rapids', 'Denver']
not_east_coast1 = df[df['city'].isin(cities_to_select3)]

east_coast1 = east_coast1.drop(columns= ['Address', 'city', 'cityId', 'county', 'countyFIPS', 'countyId', 'homeType', 'livingArea', 'regionString', 'state', 'lastSoldPrice', 'parcelId', 'zipcode', 'zpid', 'lotAreaValue', 'stateId'])
not_east_coast1 = not_east_coast1.drop(columns= ['Address', 'city', 'cityId', 'county', 'countyFIPS', 'countyId', 'homeType', 'livingArea', 'regionString', 'state', 'lastSoldPrice', 'parcelId', 'zipcode', 'zpid', 'lotAreaValue', 'stateId'])

#replace NaN values with average for rest of the chosen columns

ctr = east_coast.columns
for col in ctr:
    mean = east_coast1[col].mean()
    east_coast1[col].fillna(mean, inplace = True)


#replace NaN values with average for rest of the chosen columns

for col in ctr:
    mean = not_east_coast1[col].mean()
    not_east_coast1[col].fillna(mean, inplace = True)


X2_train = not_east_coast1.drop(columns = 'Lead')
X2_test = east_coast1.drop(columns = 'Lead')
y2_train = not_east_coast1['Lead']
y2_test = east_coast1['Lead']


model2 = xgb.XGBClassifier(**best_params)

# Train the model
model2.fit(X2_train, y2_train)

# Make predictions on the test set
y2_pred = model2.predict(X2_test)

# Calculate accuracy
accuracy2 = accuracy_score(y2_test, y2_pred)
accuracy2

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

0.8817444672505944

In [ ]:
y_1